In [0]:
# import key libraries
import pyspark
from pyspark.sql.types import IntegerType, FloatType, DateType
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime as dt
import pytz 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn import model_selection 
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.mixture import GaussianMixture
from scipy.signal import find_peaks
from sklearn.feature_selection import VarianceThreshold

def creat_sql(select_site, select_machine):
    """
    Function that create sql that allow users to interact with database
    Parameters:
    select_site: str - Site name user selected
    select_machine: str - Machine name user selected
    """
    # build machine_table 
    machine_table = '_'.join([select_site,select_machine,'timeseries'])
    # build data_table
    data_table = '.'.join(['groupdb_famc_energy_analytics',machine_table])
    # build sql 
    sql = 'select * from ' +data_table
    return sql

def create_widget(name, data, feature, type):
    """
    Function that creates widges in the notebook that allow users interact with the code
    Parameters:
    name: str - Name desired for the widget
    data: pd.DataFrame - dataframe containing the data needed to create the widget
    feature: str - name of the column to be used to extract the unique values from data
    """
    items = data[feature].unique()
    if type == 'dropdown':
        dbutils.widgets.dropdown(name, items[0], [x for x in items])
    elif type == 'multiselect':
        dbutils.widgets.multiselect(name, items[0], [x for x in items])
    else:
        print('Widget type not recognized')

def historical_data (data, n_months):
    """
    Helper function to get the historical data
    Parameters:
    data: pd.DataFrame - dataframe containing the data needed to retrieve historical data
    n_months: int - Number of months the user can track from historoical data, defaulted to be the last 18 months
    """
    today = date.today()
    
    #past_date  = today - pd.DateOffset(months=n_months)
    past_date  = today - pd.DateOffset(months=n_months)
    
    data = data[data['time_bucket_local'] >= str(past_date)]
    
    return data

def prod_codes_sorting(data):
    """
    Helper function that sort the prod_codes of one machine by Descending order via number of datapoints
    Parameters:
    data: pd.DataFrame - dataframe containing the data needed to retrieve prod_codes
    """ 
    prod_code = data.groupby('prod_code')['time_bucket_local'].count().reset_index().sort_values(by='time_bucket_local',ascending = False)
    return prod_code

def get_time_series_data (data,select_prod_codes):
    """
    Function that create training data based on the prod_codes that have been selected
    Parameters:
    data: pd.DataFrame - dataframe containing the data needed to create training data
    """ 
     # filter to only selected prod_codes
    data = data [data['prod_code'].isin(select_prod_codes)] 
    
    # get rid of redundant columns 
    time_series_data = data.iloc[:,2:]
    time_series_data.drop(['machine','time_bucket_utc'],axis = 1,inplace = True)
    
    return time_series_data   

def get_training_data (data,select_prod_codes):
    """
    Function that create training data based on the prod_codes that have been selected
    Parameters:
    data: pd.DataFrame - dataframe containing the data needed to create training data
    select_prod_codes: list - list conatining the select_prod_codes 
    """
    # filter to only selected prod_codes
    data = data [data['prod_code'].isin(select_prod_codes)] 


    # get rid of redundant columns     
    training_data = data.iloc[:,5:]     
    
    return training_data  

def remove_outlier(df):
    df = df.dropna()
    # define outlier function for each column 
    def outliers(df, ft): 
        
        lower_bound = df[ft].quantile(0.05)
        upper_bound = df[ft].quantile(0.99)
        
        ft_index = df.index[(df[ft]<lower_bound)|(df[ft]>upper_bound)]
        return ft_index
    
    remove_index =[]
    for col in df.columns:
        remove_index.extend(outliers(df, col))
        
    remove_index=sorted(set(remove_index))    
    df = df.drop(remove_index)    
    return df 

# The following section is for auto_visaulizatin DTreeReg_mix_gaussian_splits testing
def get_gmm_splits(data, variable):     
    """
    Function that splits the data base on the GMM algo. 
    Parameters:
    data: pd.DataFrame - dataframe containing the data needed to get peaks
    variable: str - the variable that for peak detection
    """ 
        
    # extract variable data 
    data = np.array(data[variable])
    # select the best n_numbers, looping from 1 to 3 >> edge case that max(data) == min (data), which means the variable is consant, then the best_n_components is defaluted to be 1 culster only.
    #compare with n_components best fit the data, ranging from 1 to 3.
    # based on the matric of gmm.bic Bayesian information criterion (BIC):This criterion gives us an estimation on how much is good the GMM in terms of predicting the data we actually have. The lower is the BIC, the better is the model to actually predict the data we have, and by extension, the true, unknown, distribution.

    if max (data) == min(data):
        best_n_components = 1
    else: 
        gmm_result = []
        for n_components in range(1,4):
            gmm = GaussianMixture(n_components).fit(data.reshape(-1, 1))
            gmm_result.append(gmm.bic(data.reshape(-1, 1)))
        best_n_components = gmm_result.index(min(gmm_result))+1
    